## Upload All File on SQL

In [3]:
import pandas as pd
import psycopg2
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('order_details.csv', 'order_detail'),
    ('./orders.csv', 'orders'),
    ('./pizza_types.csv', 'pizza_type'),
    ('./pizzas.csv', 'pizzas'),  # Added payments.csv for specific handling
]

# Folder containing the CSV files
folder_path = "C:/Users/moham/Desktop/Data Analysis/pizza_sales"

# Define a function to map pandas data types to SQL types
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INTEGER'
    elif pd.api.types.is_float_dtype(dtype):
        return 'REAL'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'TIMESTAMP'
    else:
        return 'TEXT'

try:
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        host='localhost',
        user='postgres',
        password='mohammad',
        dbname='pizza-store',
        port='5432'  # typically 5432 for PostgreSQL
    )
    cursor = conn.cursor()
    print("Connected to the database.")

    # Loop over CSV files and load each into PostgreSQL
    for csv_file, table_name in csv_files:
        file_path = os.path.join(folder_path, csv_file)

        # Read the CSV file into a pandas DataFrame
        df = pd.read_csv(file_path, encoding='utf-8', encoding_errors='ignore')


        # Replace NaN with None to handle SQL NULL
        df = df.where(pd.notnull(df), None)
        
        # Clean column names
        df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

        # Generate the CREATE TABLE statement with appropriate data types
        columns = ', '.join([f'"{col}" {get_sql_type(df[col].dtype)}' for col in df.columns])
        create_table_query = f'CREATE TABLE IF NOT EXISTS "{table_name}" ({columns})'
        cursor.execute(create_table_query)
        print(f"Table `{table_name}` created or already exists.")

        # Insert DataFrame data into the PostgreSQL table
        for _, row in df.iterrows():
            # Convert row to tuple and handle NaN/None explicitly
            values = tuple(None if pd.isna(x) else x for x in row)
            sql = f"INSERT INTO \"{table_name}\" ({', '.join(['"' + col + '"' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
            cursor.execute(sql, values)

        # Commit the transaction for the current CSV file
        conn.commit()
        print(f"Data from `{csv_file}` inserted successfully into `{table_name}`.")

except psycopg2.Error as err:
    print(f"Error: {err}")
finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("Database connection closed.")


Connected to the database.
Table `order_detail` created or already exists.
Data from `order_details.csv` inserted successfully into `order_detail`.
Table `orders` created or already exists.
Data from `./orders.csv` inserted successfully into `orders`.
Table `pizza_type` created or already exists.
Data from `./pizza_types.csv` inserted successfully into `pizza_type`.
Table `pizzas` created or already exists.
Data from `./pizzas.csv` inserted successfully into `pizzas`.
Database connection closed.
